In [1]:
import altair as alt

from deep_traffic_generation.tc_vae_dp import TCVAEDP
from deep_traffic_generation.core.datasets import TrafficDataset

from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
import numpy as np

from traffic.core.projection import EuroPP
import pandas as pd
from tqdm import tqdm

import torch

from deep_traffic_generation.core.utils import traffic_from_data

import warnings
warnings.filterwarnings("ignore")

torch.manual_seed(42)
np.random.seed(42)

In [2]:
N_SAMPLES = 10

In [3]:
dataset = TrafficDataset.from_file(
    "../deep_traffic_generation/data/simplified_15.pkl",
    features=["x", "y", "altitude", "timedelta"],
    scaler=MinMaxScaler(feature_range=(-1, 1)),
    shape="image",
)
dataset

Dataset TrafficDataset
    Number of datapoints: 3968
    File location: ../deep_traffic_generation/data/simplified_15.pkl
    MinMaxScaler(feature_range=(-1, 1))

In [4]:
model = TCVAEDP.load_from_checkpoint(
    "../deep_traffic_generation/lightning_logs/tc_vae_dp/version_10/checkpoints/epoch=300-step=902.ckpt",
    hparams_file="../deep_traffic_generation/lightning_logs/tc_vae_dp/version_10/hparams.yaml",
    dataset_params=dataset.parameters,
)
model.eval()

TCVAEDP(
  (encoder): TCEncoder(
    (encoder): Sequential(
      (0): TCN(
        (network): Sequential(
          (0): ResidualBlock(
            (tmp_block1): TemporalBlock(
              (conv): Conv1d(4, 16, kernel_size=(3,), stride=(1,))
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (tmp_block2): TemporalBlock(
              (conv): Conv1d(16, 16, kernel_size=(3,), stride=(1,))
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (downsample): Conv1d(4, 16, kernel_size=(1,), stride=(1,))
          )
          (1): ResidualBlock(
            (tmp_block1): TemporalBlock(
              (conv): Conv1d(16, 16, kernel_size=(3,), stride=(1,), dilation=(2,))
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (tmp_block2): TemporalBlock(
              (conv): Conv1d(16, 16, kernel_size=(3,), stride=(1,), dilation=(2,))
              (dropout): Dropout(p=0.0, inplace=False)
            )
         

In [5]:
z, _ = model.encoder(torch.Tensor(dataset.data), [])
x = z.detach().cpu().numpy()
# z_scale = np.exp(z_scale.detach().cpu().numpy() / 2)
x.shape

(3968, 16)

In [6]:
original, l, info = dataset[0]
_, _, reconstruction = model(original.unsqueeze(0), l)

data = np.concatenate((original.unsqueeze(0).numpy(), reconstruction.detach().numpy()))
data = np.transpose(data, (0, 2, 1))
data.shape

(2, 14, 4)

In [8]:
data = data.reshape(-1, 4)
data = dataset.scaler.inverse_transform(data)
data.shape

(28, 4)